In [104]:
import pandas as pd
# from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup, Comment
from time import sleep
import re
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [29]:
df = pd.read_csv('allActive_raw.csv')

In [30]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'D.C.': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [31]:
df['abbrev'] = df['state'].map(us_state_abbrev)

In [32]:
exists = []

In [33]:
for index, row in df.iterrows():
    citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    if req.status_code == 404:
        url = 'https://www.maxpreps.com/high-schools/'+citystring+'/basketball/schedule.htm'
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        req = session.get(url, headers=hdr)
        if req.status_code == 404:
            exists.append('false')
        else:
            exists.append('true')
    else:
        exists.append('true')
    print(citystring + str(req.status_code))
        

ackerman-indians-(ackerman,ms)200
adair-county-indians-(columbia,ky)200
albany-indians-(albany,ga)404
altamont-indians-(altamont,il)200
alvarado-indians-(alvarado,tx)200
amesbury-indians-(amesbury,ma)200
anacoco-indians-(anacoco,la)200
anacostia-indians-(washington,dc)200
andale-indians-(andale,ks)200
anderson-indians-(anderson,in)200
arapaho-indians-(arapaho,ok)200
archbishop-ryan-indian-raiders-(philadelphia,pa)200
arickaree-indians-(anton,co)200
armuchee-indians-(rome,ga)200
asher-indians-(asher,ok)200
athens-indians-(athens,mi)200
avinger-indians-(avinger,tx)200
baldwin-indians-(baldwin,fl)200
bartlett-indians-(webster,ma)200
bartlett/douglas-indians-(webster,ma)404
berkeley-springs-indians-(berkeley-springs,wv)200
berlin-indians-(berlin,wi)200
bern-indians-(bern,ks)404
billerica-memorial-indians-(billerica,ma)200
biloxi-indians-(biloxi,ms)200
bismarck-indians-(bismarck,mo)200
blacksburg-indians-(blacksburg,va)200
brentwood-indians-(brentwood,ny)200
bridgeport-indians-(bridgeport,w

In [55]:
# del exists[0]
len(exists)

1455

In [34]:
df['exists'] = exists

In [35]:
df.to_csv('allActive.csv')

In [97]:
df_false = df[df['exists'] == 'false']

In [98]:
df_false

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists
2,23,Albany Indians,Albany,Georgia,active,/teams/albany-indians,NaN,31.578206,-84.155681,GA,false
19,148,Bartlett/Douglas Indians,Webster,Massachusetts,active,/teams/bartlettdouglas-indians,NaN,42.054184,-71.849818,MA,false
22,195,Bern Indians,Bern,Kansas,active,/teams/bern-indians,NaN,39.962222,-95.971942,KS,false
54,454,Cheyenne Central Indians,Cheyenne,Wyoming,active,/teams/cheyenne-central-indians,NaN,41.139981,-104.820246,WY,false
59,478,Christian Academy of Indiana Warriors,New Albany,Indiana,active,/teams/christian-academy-indiana-warriors,NaN,38.285625,-85.824131,IN,false
...,...,...,...,...,...,...,...,...,...,...,...
1407,1085,James Caldwell Chiefs,West Caldwell,New Jersey,active,/teams/james-caldwell-chiefs,NaN,40.840933,-74.301815,NJ,false
1426,1590,Nonnewaug/Shepaug Valley Chiefs,Woodbury,Connecticut,active,/teams/nonnewaugshepaug-valley-chiefs,NaN,41.544540,-73.209002,CT,false
1434,1809,Plymouth-Canton Educational Park Chiefs,Canton,Michigan,active,/teams/plymouth-canton-educational-park-chiefs,NaN,42.307655,-83.486896,MI,false
1435,1895,Red Cloud Chiefs,Red Cloud,South Dakota,active,/teams/red-cloud-chiefs,NaN,NaN,NaN,SD,false


,Unnamed: 0,Unnamed: 0.1,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists
0,0,3,Ackerman Indians,Ackerman,Mississippi,active,/teams/ackerman-indians,NaN,33.310124,-89.172843,MS,True
1,1,7,Adair County Indians,Columbia,Kentucky,active,/teams/adair-county-indians,NaN,37.102841,-85.306350,KY,True
2,2,23,Albany Indians,Albany,Georgia,active,/teams/albany-indians,NaN,31.578206,-84.155681,GA,False
3,3,39,Altamont Indians,Altamont,Illinois,active,/teams/altamont-indians,NaN,39.062036,-88.748122,IL,True
4,4,44,Alvarado Indians,Alvarado,Texas,active,/teams/alvarado-indians,NaN,32.409557,-97.210288,TX,True
...,...,...,...,...,...,...,...,...,...,...,...,...
1450,1450,2642,West Ouachita Chiefs,West Monroe,Louisiana,active,/teams/west-ouachita-chiefs,NaN,32.518478,-92.147635,LA,True
1451,1451,2714,White Pigeon Chiefs,White Pigeon,Michigan,active,/teams/white-pigeon-chiefs,NaN,41.798106,-85.643323,MI,True
1452,1452,2779,Wisconsin Dells Chiefs,Wisconsin Dells,Wisconsin,active,/teams/wisconsin-dells-chiefs,NaN,43.625617,-89.771565,WI,True
1453,1453,2813,Wyoming Indian Chiefs,Ethete,Wyoming,active,/teams/wyoming-indian-chiefs,NaN,43.005564,-108.727620,WY,True


In [14]:
exists = []
year = []
for index, row in df.iterrows():
    try:
        citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    except:
        exists.append('false')
        year.append('false')
        continue
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    year = ''
    if req.status_code == 404:
        url = 'https://www.maxpreps.com/high-schools/'+citystring+'/basketball/schedule.htm'
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        req = session.get(url, headers=hdr)
        soup = BeautifulSoup(req.text)
        try:
            year = soup.find(id='ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Team').text
            if "2020" in year:
                exists.append('true')
            else:
                exists.append('false')
        except:
            exists.append('false')
    else:
        soup = BeautifulSoup(req.text)
        try:
            year = soup.find(id='ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Team').text
            if "2020" in year:
                exists.append('true')
            else:
                exists.append('false')
        except:
            exists.append('false')
    print(citystring + str(year))

ackerman-indians-(ackerman,ms)Varsity 2020
adair-county-indians-(columbia,ky)Varsity 2020
altamont-indians-(altamont,il)Varsity 2020-21
alvarado-indians-(alvarado,tx)Varsity 2020
amesbury-indians-(amesbury,ma)Varsity 2020
anacoco-indians-(anacoco,la)Varsity 2020-21
anacostia-indians-(washington,dc)Varsity 2020
andale-indians-(andale,ks)Varsity 2020
anderson-indians-(anderson,in)Varsity 2020
arapaho-indians-(arapaho,ok)Varsity 2020-21
archbishop-ryan-indian-raiders-(philadelphia,pa)Varsity 2020
arickaree-indians-(anton,co)Varsity 2016
armuchee-indians-(rome,ga)Varsity 2020
asher-indians-(asher,ok)Varsity 2020-21
athens-indians-(athens,mi)Varsity 2020
avinger-indians-(avinger,tx)Varsity 2020-21
baldwin-indians-(baldwin,fl)Varsity 2020
bartlett-indians-(webster,ma)Varsity 2020
bartlett-indians-(webster,ma)Varsity 2020
berkeley-springs-indians-(berkeley-springs,wv)Varsity 2020
berlin-indians-(berlin,wi)Varsity 2020
billerica-memorial-indians-(billerica,ma)Varsity 2020
biloxi-indians-(bilox

In [16]:
df['exists'] = exists

In [72]:
df_false = df[df['exists'] == 'false']
df_true = df[df['exists'] == 'true']

In [24]:
df_true.to_csv('allTrue.csv')

In [27]:
df_false.to_csv('allFalse.csv')

In [53]:
allActiveFx = pd.read_csv('allActive_2.csv')

In [89]:
absentOnes = df_false[df_false['team_name'].isin(allActiveFx['team_name'])].dropna()

In [90]:
absentOnes

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists
561,242,BOLD Warriors,Olivia,Minnesota,active,/teams/bold-warriors,Bird Island-Olivia-Lake Lillian District,44.776350,-94.989721,MN,false
754,976,Henry Sibley Warriors,Mendota Heights,Minnesota,active,/teams/henry-sibley-warriors,Henry Sibley High School was founded in 1954. ...,44.883577,-93.138275,MN,false


In [51]:
absentOnes[absentOnes['state'] == 'Ohio']

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists


In [79]:
allActiveFx[allActiveFx['team_name'] == 'Whitney M. Young Warriors']

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev
1151,2724,Whitney M. Young Warriors,Cleveland,Ohio,active,/teams/whitney-m-young-warriors,NaN,41.505161,-81.693445,OH


In [93]:
df_true = df[df['exists'] == 'true']

In [102]:
df_false = df_false[df_false.columns.drop(['exists'])]
df_false

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev
2,23,Albany Indians,Albany,Georgia,active,/teams/albany-indians,NaN,31.578206,-84.155681,GA
19,148,Bartlett/Douglas Indians,Webster,Massachusetts,active,/teams/bartlettdouglas-indians,NaN,42.054184,-71.849818,MA
22,195,Bern Indians,Bern,Kansas,active,/teams/bern-indians,NaN,39.962222,-95.971942,KS
54,454,Cheyenne Central Indians,Cheyenne,Wyoming,active,/teams/cheyenne-central-indians,NaN,41.139981,-104.820246,WY
59,478,Christian Academy of Indiana Warriors,New Albany,Indiana,active,/teams/christian-academy-indiana-warriors,NaN,38.285625,-85.824131,IN
...,...,...,...,...,...,...,...,...,...,...
1407,1085,James Caldwell Chiefs,West Caldwell,New Jersey,active,/teams/james-caldwell-chiefs,NaN,40.840933,-74.301815,NJ
1426,1590,Nonnewaug/Shepaug Valley Chiefs,Woodbury,Connecticut,active,/teams/nonnewaugshepaug-valley-chiefs,NaN,41.544540,-73.209002,CT
1434,1809,Plymouth-Canton Educational Park Chiefs,Canton,Michigan,active,/teams/plymouth-canton-educational-park-chiefs,NaN,42.307655,-83.486896,MI
1435,1895,Red Cloud Chiefs,Red Cloud,South Dakota,active,/teams/red-cloud-chiefs,NaN,NaN,NaN,SD


In [103]:
allActiveFx

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev
0,3,Ackerman Indians,Ackerman,Mississippi,active,/teams/ackerman-indians,NaN,33.310124,-89.172843,MS
1,7,Adair County Indians,Columbia,Kentucky,active,/teams/adair-county-indians,NaN,37.102841,-85.306350,KY
2,39,Altamont Indians,Altamont,Illinois,active,/teams/altamont-indians,NaN,39.062036,-88.748122,IL
3,44,Alvarado Indians,Alvarado,Texas,active,/teams/alvarado-indians,NaN,32.409557,-97.210288,TX
4,48,Amesbury Indians,Amesbury,Massachusetts,active,/teams/amesbury-indians,NaN,42.857954,-70.930092,MA
...,...,...,...,...,...,...,...,...,...,...
1405,2571,Waukomis Chiefs,Waukomis,Oklahoma,active,/teams/waukomis-chiefs,NaN,36.280313,-97.898113,OK
1406,2642,West Ouachita Chiefs,West Monroe,Louisiana,active,/teams/west-ouachita-chiefs,NaN,32.518478,-92.147635,LA
1407,2714,White Pigeon Chiefs,White Pigeon,Michigan,active,/teams/white-pigeon-chiefs,NaN,41.798106,-85.643323,MI
1408,2779,Wisconsin Dells Chiefs,Wisconsin Dells,Wisconsin,active,/teams/wisconsin-dells-chiefs,NaN,43.625617,-89.771565,WI


In [124]:
absentOnes = pd.merge(df_false, allActiveFx, on=['team_name','city'], how='outer', indicator=True)

In [125]:
absentOnes['Exist'] = np.where(absentOnes['_merge'] == 'both', True, False)

In [126]:
absentOnes = absentOnes[absentOnes['Exist'] == True]

In [130]:
absentOnes = absentOnes[['Unnamed: 0_x','team_name','city','state_x','status_x','url_y','tidbit_y','lat_x','long_x','abbrev_x','Exist']]

In [134]:
absentOnes.columns = ['Unnamed: 0','team_name','city','state','status','url','tidbit','lat','long','abbrev','exists']

In [128]:
df_true.head()

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists
0,3,Ackerman Indians,Ackerman,Mississippi,active,/teams/ackerman-indians,NaN,33.310124,-89.172843,MS,true
1,7,Adair County Indians,Columbia,Kentucky,active,/teams/adair-county-indians,NaN,37.102841,-85.306350,KY,true
3,39,Altamont Indians,Altamont,Illinois,active,/teams/altamont-indians,NaN,39.062036,-88.748122,IL,true
4,44,Alvarado Indians,Alvarado,Texas,active,/teams/alvarado-indians,NaN,32.409557,-97.210288,TX,true
5,48,Amesbury Indians,Amesbury,Massachusetts,active,/teams/amesbury-indians,NaN,42.857954,-70.930092,MA,true


In [135]:
final_schools = pd.concat([df_true,absentOnes])

In [141]:
final_schools = final_schools.drop_duplicates()

In [142]:
final_schools.to_csv('finalData.csv')

In [146]:
exists = []
year_array = []
for index, row in final_schools.iterrows():
    try:
        citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    except:
        exists.append('noString')
        year_array.append('noString')
        continue
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    if req.status_code == 404:
        url = 'https://www.maxpreps.com/high-schools/'+citystring+'/basketball/schedule.htm'
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        req = session.get(url, headers=hdr)
        if req.status_code == 404:
            exists.append('false')
            year_array.append('404 Error')
        else:
            soup = BeautifulSoup(req.text)
            try:
                year = soup.find(id='ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Team').text
                if "2020" in year:
                    exists.append('true')
                    year_array.append('2020')
                else:
                    exists.append('true')
                    year_array.append('closed')
            except:
                exists.append('true')
                year_array.append('noYear')
    else:
        soup = BeautifulSoup(req.text)
        try:
            year = soup.find(id='ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Team').text
            if "2020" in year:
                exists.append('true')
                year_array.append('2020')
            else:
                exists.append('true')
                year_array.append('closed')
        except:
            exists.append('true')
            year_array.append('noYear')
    print(citystring)

ackerman-indians-(ackerman,ms)
adair-county-indians-(columbia,ky)
altamont-indians-(altamont,il)
alvarado-indians-(alvarado,tx)
amesbury-indians-(amesbury,ma)
anacoco-indians-(anacoco,la)
anacostia-indians-(washington,dc)
andale-indians-(andale,ks)
anderson-indians-(anderson,in)
arapaho-indians-(arapaho,ok)
archbishop-ryan-indian-raiders-(philadelphia,pa)
arickaree-indians-(anton,co)
armuchee-indians-(rome,ga)
asher-indians-(asher,ok)
athens-indians-(athens,mi)
avinger-indians-(avinger,tx)
baldwin-indians-(baldwin,fl)
bartlett-indians-(webster,ma)
berkeley-springs-indians-(berkeley-springs,wv)
berlin-indians-(berlin,wi)
billerica-memorial-indians-(billerica,ma)
biloxi-indians-(biloxi,ms)
bismarck-indians-(bismarck,mo)
blacksburg-indians-(blacksburg,va)
brentwood-indians-(brentwood,ny)
bridgeport-indians-(bridgeport,wv)
brimfield-indians-(brimfield,il)
broken-bow-indians-(broken-bow,ne)
browning-indians-(browning,mt)
buhl-indians-(buhl,id)
burroughs-indians-(burbank,ca)
byhalia-indians-

In [148]:
final_schools['exists'] = exists

<ipython-input-148-20f3b0582d56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_schools['exists'] = exists


In [150]:
final_schools['year'] = year_array

<ipython-input-150-bb67e0fdd5a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_schools['year'] = year_array


In [159]:
final_schools[final_schools['year'] == 'closed']

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,year
12,86.0,Arickaree Indians,Anton,Colorado,active,/teams/arickaree-indians,NaN,39.741650,-103.217162,CO,true,closed
25,223.0,Bismarck Indians,Bismarck,Missouri,active,/teams/bismarck-indians,NaN,37.769217,-90.624848,MO,true,closed
35,327.0,Caddo Hills Indians,Norman,Arkansas,active,/teams/caddo-hills-indians,NaN,34.455100,-93.680191,AR,true,closed
41,368.0,Candor Indians,Candor,New York,active,/teams/candor-indians,NaN,42.232902,-76.342419,NY,true,closed
45,393.0,Carrollton Indians,Carrollton,Alabama,active,/teams/carrollton-indians,NaN,33.261787,-88.095026,AL,true,closed
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,387.0,Carlisle Lady Chiefs,Martinsville,Virginia,active,/teams/carlisle-lady-chiefs,NaN,36.691526,-79.872539,VA,true,closed
1396,554.0,Cornell Chiefs,Cornell,Wisconsin,active,/teams/cornell-chiefs,NaN,45.167188,-91.149311,WI,true,closed
1405,1065.0,Iroquois Chiefs,Iroquois,South Dakota,active,/teams/iroquois-chiefs-0,NaN,44.366920,-97.850917,SD,true,closed
1425,1589.0,Nonnewaug Chiefs,Woodbury,Connecticut,active,/teams/nonnewaug-chiefs,NaN,41.544540,-73.209002,CT,true,closed


In [160]:
warriors_active = final_schools[final_schools['team_name'].str.contains("Warriors", na=False)]

In [161]:
warriors_active

,Unnamed: 0,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,year
158,1039.0,Indian Land Warriors,Fort Mill,South Carolina,active,/teams/indian-land-warriors,NaN,35.007370,-80.945076,SC,true,2020
162,1043.0,Indian River Warriors,Philadelphia,New York,active,/teams/indian-river-warriors,NaN,44.154622,-75.708847,NY,true,2020
168,1051.0,Indianola Warriors,Indianola,Oklahoma,active,/teams/indianola-warriors,NaN,35.162876,-95.774715,OK,true,2020
380,2255.0,St. Francis Indian Warriors,St. Francis,South Dakota,active,/teams/st-francis-indian-warriors,NaN,43.144450,-100.905147,SD,true,2020
514,2.0,Abundant Life Academy Warriors,Kanab,Utah,active,/teams/abundant-life-academy-warriors,NaN,37.048142,-112.528548,UT,true,closed
...,...,...,...,...,...,...,...,...,...,...,...,...
94,2582.0,Waxahachie Preparatory Academy Warriors,Waxahachie,Texas,active,/teams/waxahachie-preparatory-academy-warriors,NaN,32.394491,-96.843936,TX,false,404 Error
97,2675.0,Westland Lutheran Warriors,Westland,Michigan,active,/teams/westland-lutheran-warriors,NaN,42.323806,-83.400532,MI,false,404 Error
98,2705.0,Wheaton/Herman-Norcross Warriors,Wheaton,Minnesota,active,/teams/wheatonherman-norcross-warriors,NaN,45.804405,-96.499233,MN,false,404 Error
100,2724.0,Whitney M. Young Warriors,Cleveland,Ohio,active,/teams/whitney-m-young-warriors,NaN,41.505161,-81.693445,OH,false,404 Error
